In [ ]:
### Group 1 
### Data for final project

## EFTs

### Nikkei 225 Stock Average
This is Japan's primary stock index. To evaluate trading strategies based on the index, we'll be looking at ETFs that track the index. 

There are 2 primary ETFs for the index:$^1$ 
- iShares Core Nikkei 225 ETF (ticker: `1329.T`)
- NEXT FUNDS Nikkei 225 Exchange Traded Fund | Nomura (ticker: `1321.T`)

The index on Yahoo Finance is: 
`^N225`

The Nikkei 255 ETF by Normura has 10x as much volume as the iShares Core Nikkei, but the iShares Core Nikkei costs half in management fees incl. taxes (`1329.T` fees: 0.0495%$^2$ versus `1321.T` fees: 0.11%$^3$)

Although we'd ordinarily choose `1329.T` for the lower management fees, it is denominatedin YEN. Thus, we will also pull data for `EWJ`, which gives investors broad exposure to the Japanese market

*Sources:*
- $^1$ https://www.investopedia.com/articles/investing/102114/how-invest-nikkei-225.asp
- $^2$ https://www.blackrock.com/jp/individual-en/en/literature/fact-sheet/1329-ishares-core-nikkei-225-etf-fund-fact-sheet-en-jp.pdf
- $^3$ https://global.nomura-am.co.jp/nextfunds/products/1321_TPros.pdf

### European Performance

There are several European ETFs we could use as proxy for the Eurozone's health. However, for the purposes of this analysis, we will use the Stoxx 50, which is designed to represent the 50 largest Eurozone companies.$^1$ Following this index is advantegous as it allows us to have exposure to Belgium, Finland, France, Germany, Ireland, Italy, the Netherlands, and Spain. The ticker for the index is `^STOXX50E`

One of the most popular ETFs that tracks this index is the SPDR EURO STOXX 50 ETF, `FEZ`. Another available EFT is iShares Core EURO STOXX 50 UCITS ETF `EUE.MI`

For simplicity, we will use the `FEZ`, as its prices are already in USD. 

*Sources:*
- $^1$ https://www.investopedia.com/terms/d/dowjoneseurostoxx50.asp

### Pulling the data

In [4]:
import pandas as pd
import numpy as np

import yfinance as yf
import pandas_datareader as pdr

In [29]:
STARTDATE = "1995-01-01"
ENDDATE = "2024-02-29"

FILEOUT = '../Final Project/etf_euro_jap_project_data.xlsx'

In [30]:
TICKS = [
    '1329.T',
    '1321.T',
    'FEZ',
    '^N225',
    '^STOXX50E'
]

FLDS = ['quoteType','shortName','volume','totalAssets','trailingAnnualDividendYield','longBusinessSummary']

In [31]:
info = pd.DataFrame(index=TICKS,columns=FLDS)
info.index.name = 'ticker'
for tick in info.index:
    temp = yf.Ticker(tick).get_info()

    for fld in FLDS:
        if fld in temp.keys():
            info.loc[tick,fld] = temp[fld]            

In [32]:
prices = yf.download(TICKS, start=STARTDATE, end=ENDDATE)['Adj Close'][info.index]

prices.index.name = 'date'

prices.dropna(inplace=True)
rets = prices.pct_change().dropna()

[*********************100%%**********************]  5 of 5 completed


In [33]:
info

,quoteType,shortName,volume,totalAssets,trailingAnnualDividendYield,longBusinessSummary
ticker,,,,,,
1329.T,ETF,BLACKROCK JAPAN CO LTD ISHARES,24042,1348756045824,NaN,NaN
1321.T,ETF,NOMURA ASSET MANAGEMENT CO LTD,328159,10899299500032,NaN,NaN
FEZ,ETF,SPDR DJ Euro STOXX 50 Etf,768955,3591639040,0.015321,"The fund employs a sampling strategy, which me..."
^N225,INDEX,Nikkei 225,NaN,NaN,NaN,NaN
^STOXX50E,INDEX,ESTX 50 PR.EUR,NaN,NaN,NaN,NaN


In [34]:
rets

Ticker,1329.T,1321.T,FEZ,^N225,^STOXX50E
date,,,,,
2009-01-06,0.002205,0.001088,0.008520,0.004171,0.009799
2009-01-07,0.017602,0.020652,-0.013691,0.017443,-0.015467
2009-01-08,-0.028108,-0.040468,0.010041,-0.039269,-0.008662
2009-01-09,0.005562,-0.002220,-0.044737,-0.004464,-0.011909
2009-01-13,-0.066372,-0.048943,-0.041016,-0.047856,-0.030286
...,...,...,...,...,...
2024-02-21,-0.003271,-0.002762,0.006487,-0.002644,0.003157
2024-02-22,0.021964,0.021909,0.015911,0.021863,0.016763
2024-02-26,0.002223,0.001971,0.001784,0.003454,0.001839


In [35]:
with pd.ExcelWriter(FILEOUT) as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    rets.to_excel(writer, sheet_name='total returns')
    prices.to_excel(writer, sheet_name='prices')

## Interest Rates

For interest rates, we'll use the FRED website.

**Japanese Interest Rates**
We weren't able to find interest rate information for free from the BOJ, so we'll be using the Long-term Gov Bond Yields (10-Year) for the Japanese data. The corresponding tickler is `IRLTLT01JPM156N`

For completeness, we'll also pull Interest Rates: Immediate Rates (< 24 Hours): Central Bank Rates, tickler `IRSTCB01JPM156N`

**Eurozeon Interest Rates**
We'll pull in the ECB rates and the long-term gov bond yields across 19 Eurozone countries. 

**Ticklers**: 
1. Interest Rates: Long-Term Government Bond Yields: 10-Year: Main (Including Benchmark) for Euro Area (19 Countries) | `IRLTLT01EZM156N`
2. ECB Deposit Facility Rate for Euro Area: `ECBDFR`
3. ECB Main Refinancing Operations Rate: Fixed Rate Tenders for Euro Area: `ECBMRRFR`
4. ECB Marginal Lending Facility Rate for Euro Area: `ECBMLFR`

In [36]:
SAVE_DATA = True

startdate = pd.to_datetime(STARTDATE)
enddate = pd.to_datetime(ENDDATE)

ticks = ['IRLTLT01JPM156N','IRSTCB01JPM156N','IRLTLT01EZM156N','ECBDFR', 'ECBMRRFR', 'ECBMLFR']

rates = pdr.DataReader(ticks,'fred',startdate,enddate)
rates.index.name = 'date'
rates.index = pd.to_datetime(rates.index)

In [37]:
info = pd.DataFrame(index=ticks, data=['BOJ 10-year','BOJ 24h Rates','Euro 10-Year', 'ECB Deposit Rate',
                                       'ECB Refinancing', 'ECB Marginal Rate'])
info.index.name = 'ticker (FRED)'
info.columns = ['description']
info = info.reset_index().set_index('description')
info

,ticker (FRED)
description,
BOJ 10-year,IRLTLT01JPM156N
BOJ 24h Rates,IRSTCB01JPM156N
Euro 10-Year,IRLTLT01EZM156N
ECB Deposit Rate,ECBDFR
ECB Refinancing,ECBMRRFR
ECB Marginal Rate,ECBMLFR


In [38]:
if SAVE_DATA:
    outfile = '../Final Project/euro_japan_rates.xlsx'
    with pd.ExcelWriter(outfile) as writer:  
        info.to_excel(writer, sheet_name= 'info', index=True)
        rates.to_excel(writer, sheet_name= 'data', index=True)

## Economic Data
We will use the following economic indicators: 
-   Inflation, consumer prices for Japan: `FPCPITOTLZGJPN`, Percent, Not Seasonally Adjusted, Annual Frequency
-  Inflation, consumer prices for the Euro Area: `FPCPITOTLZGEMU`, Percent, Not Seasonally Adjusted, Annual Frequency
-  Real Gross Domestic Product (Euro/ECU Series) for Euro Area (19 Countries): `CLVMEURSCAB1GQEA19`, Millions of Chained 2010 Euros, Seasonally Adjusted, Quarterly Data
- Real Gross Domestic Product for Japan: `JPNRGDPEXP`, Billions of Chained 2015 Yen, Seasonally Adjusted, Quarterly

In [39]:
ticks = ['FPCPITOTLZGJPN','FPCPITOTLZGEMU','CLVMEURSCAB1GQEA19','JPNRGDPEXP']
labels = ['CPI Japan Annual','CPI Euro Annual','RGDP Euro','RGDP Japan']

data = pdr.DataReader(ticks,'fred',startdate,enddate)
data.index.name = 'date'
data.index = pd.to_datetime(data.index)
data.columns = labels

In [40]:
info = pd.DataFrame(index=ticks, data=labels)
info.index.name = 'ticker (FRED)'
info.columns = ['description']
info = info.reset_index().set_index('description')
info

,ticker (FRED)
description,
CPI Japan Annual,FPCPITOTLZGJPN
CPI Euro Annual,FPCPITOTLZGEMU
RGDP Euro,CLVMEURSCAB1GQEA19
RGDP Japan,JPNRGDPEXP


In [42]:
if SAVE_DATA:
    outfile = '../Final Project/economic_data_eu_jap.xlsx'
    with pd.ExcelWriter(outfile) as writer:  
        info.to_excel(writer, sheet_name= 'info', index=True)
        data.to_excel(writer, sheet_name= 'data', index=True)

**NOTE**
Because this data is in YEN and Euros, we have decided to use data from the IMF, normalized into USD. We could convert into USD ourselves using the market rate, but we wanted to ensure good quality data going into the modeling. 